# 🌊 Neural Network Training for Nanofluid Prediction (Google Colab)

This notebook is optimized for **Google Colab**. Follow the setup instructions below.

## 📤 Step 1: Upload Your Data

You have **3 options**:

### Option A: Upload Files Manually
1. Click the **folder icon** 📁 in the left sidebar
2. Create folder structure: `data/processed/`
3. Upload `train_dataset.csv` and `test_dataset.csv`

### Option B: Mount Google Drive (Recommended)
Run the cell below and follow prompts

### Option C: Upload from GitHub
Clone the repo directly (see Alternative Setup below)

In [ ]:
# OPTION B: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Upload your NANO-FLUIDS folder to Google Drive first
# Then uncomment and modify this path:
# %cd /content/drive/MyDrive/NANO-FLUIDS

In [ ]:
# OPTION C: Clone from GitHub
!git clone https://github.com/BhaveshBytess/NANOFLUIDS.git
%cd NANOFLUIDS

## 🔧 Step 2: Install Dependencies

In [ ]:
# Install required packages (most are pre-installed in Colab)
!pip install -q torch scikit-learn pandas numpy matplotlib seaborn
print("✅ Dependencies installed!")

## 📊 Step 3: Load Data and Train Model

In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ========================================
# CONFIGURE PATHS BASED ON YOUR SETUP
# ========================================

# If you cloned from GitHub (Option C):
DATA_DIR = Path("data/processed")
ARTIFACT_DIR = Path("models")

# If you uploaded manually (Option A):
# DATA_DIR = Path("/content/data/processed")
# ARTIFACT_DIR = Path("/content/models")

# If using Google Drive (Option B):
# DATA_DIR = Path("/content/drive/MyDrive/NANO-FLUIDS/data/processed")
# ARTIFACT_DIR = Path("/content/drive/MyDrive/NANO-FLUIDS/models")

TRAIN_PATH = DATA_DIR / "train_dataset.csv"
TEST_PATH = DATA_DIR / "test_dataset.csv"
FEATURES = ["M", "S", "K", "phi1", "phi2", "Ec", "Pr", "eta"]
TARGETS = ["f3", "f5"]
RANDOM_STATE = 42

torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

# Verify files exist
if not TRAIN_PATH.exists():
    print(f"❌ ERROR: {TRAIN_PATH} not found!")
    print(f"Current directory: {Path.cwd()}")
    print(f"Files in current directory:")
    !ls -la
else:
    print(f"✅ Found training data at {TRAIN_PATH}")

In [ ]:
# Load data
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

X_full = train_df[FEATURES].to_numpy(dtype=float)
y_full = train_df[TARGETS].to_numpy(dtype=float)
X_test = test_df[FEATURES].to_numpy(dtype=float)
y_test = test_df[TARGETS].to_numpy(dtype=float)

X_train, X_val, y_train, y_val = train_test_split(
    X_full,
    y_full,
    test_size=0.2,
    random_state=RANDOM_STATE,
    shuffle=True,
)

print(f"✅ Data loaded successfully!")
print(f"  Train: {X_train.shape}")
print(f"  Val:   {X_val.shape}")
print(f"  Test:  {X_test.shape}")

## 🤖 Step 4: Define Neural Network Model

In [ ]:
class NanofluidDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class NanofluidNet(nn.Module):
    def __init__(self, input_dim=8, hidden1=64, hidden2=128, hidden3=64, output_dim=2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, hidden3)
        self.fc4 = nn.Linear(hidden3, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

print("✅ Model architecture defined")

## 🏋️ Step 5: Train the Model

In [ ]:
# Normalize data
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train_norm = (X_train - X_mean) / X_std
X_val_norm = (X_val - X_mean) / X_std
X_test_norm = (X_test - X_mean) / X_std

# Create datasets and dataloaders
train_dataset = NanofluidDataset(X_train_norm, y_train)
val_dataset = NanofluidDataset(X_val_norm, y_val)
test_dataset = NanofluidDataset(X_test_norm, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize model
model = NanofluidNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("✅ Training setup complete")
print(f"  Model parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# Training loop
epochs = 200
patience = 40
best_val_loss = float('inf')
patience_counter = 0

train_losses = []
val_losses = []

print("🏋️ Starting training...")
for epoch in range(epochs):
    # Training
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), ARTIFACT_DIR / "neural_network.pt")
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    if patience_counter >= patience:
        print(f"\n✅ Early stopping at epoch {epoch+1}")
        break

print(f"\n🎉 Training complete! Best val loss: {best_val_loss:.4f}")

## 📊 Step 6: Evaluate Model

In [ ]:
# Load best model
model.load_state_dict(torch.load(ARTIFACT_DIR / "neural_network.pt"))
model.eval()

# Predict on test set
y_pred = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        y_pred.append(outputs.cpu().numpy())

y_pred = np.vstack(y_pred)

# Calculate metrics
mae_f3 = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
mae_f5 = mean_absolute_error(y_test[:, 1], y_pred[:, 1])
rmse_f3 = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
rmse_f5 = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))
r2_f3 = r2_score(y_test[:, 0], y_pred[:, 0])
r2_f5 = r2_score(y_test[:, 1], y_pred[:, 1])

print("📊 TEST SET RESULTS:")
print("="*50)
print(f"f3 (velocity gradient):")
print(f"  MAE:  {mae_f3:.4f}")
print(f"  RMSE: {rmse_f3:.4f}")
print(f"  R²:   {r2_f3:.4f}")
print(f"\nf5 (temperature gradient):")
print(f"  MAE:  {mae_f5:.4f}")
print(f"  RMSE: {rmse_f5:.4f}")
print(f"  R²:   {r2_f5:.4f}")

# Save metadata
metadata = {
    'normalization': {
        'mean': X_mean.tolist(),
        'std': X_std.tolist()
    },
    'metrics': {
        'test_mae_f3': float(mae_f3),
        'test_mae_f5': float(mae_f5),
        'test_rmse_f3': float(rmse_f3),
        'test_rmse_f5': float(rmse_f5),
        'test_r2_f3': float(r2_f3),
        'test_r2_f5': float(r2_f5)
    }
}

with open(ARTIFACT_DIR / "neural_network_metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n✅ Model saved to:", ARTIFACT_DIR)

## 📥 Step 7: Download Trained Model

Download the model files to use them locally:

In [ ]:
from google.colab import files

files.download(str(ARTIFACT_DIR / "neural_network.pt"))
files.download(str(ARTIFACT_DIR / "neural_network_metadata.json"))

print("✅ Files ready for download!")